# CT scan OF THE TMJ

In [35]:
!pip install accelerate

import numpy as np
import albumentations
import random
import torch
import os

from pycocotools.coco import COCO
from albumentations.pytorch.transforms import ToTensorV2
from accelerate import Accelerator
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from matplotlib import pyplot as plt
from os.path import join as pjoin

from MaskCreator import MaskCreator
from TMJDataset import TMJDataset

In [36]:
def seed_everything(seed: int = 314159, torch_deterministic: bool = False) -> None:
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.use_deterministic_algorithms(torch_deterministic)


seed_everything(42, torch_deterministic=False)

In [37]:
annFile = 'annotations.json'
coco = COCO(annFile)
image_shapes = {img['id']: (img['height'], img['width']) for img in coco.loadImgs(coco.getImgIds())}

maskCreator = MaskCreator(coco=coco, image_shapes=image_shapes)
maskCreator.save_all_masks("Masks_npy")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Processing images: 100%|██████████| 157/157 [00:00<00:00, 701.36it/s]

All masks saved in Masks_npy


In [38]:
IMAGE_SIZE = 512  # Размер изображения

transforms = albumentations.Compose(
    [
        albumentations.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0),
        albumentations.ChannelShuffle(p=0.5),
        albumentations.AdvancedBlur(p=0.5),
        albumentations.GaussNoise(p=0.5),
        albumentations.RandomBrightnessContrast(p=0.5),
        albumentations.RandomGamma(p=0.5),
        albumentations.ColorJitter(p=0.5),
        albumentations.Normalize(),
        ToTensorV2(),
    ]
)

In [45]:
dataset = TMJDataset(image_dir="Dataset", mask_dir="Masks_npy", transform=transforms)

error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
